In [1]:
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F


In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.lin1 = nn.Linear(512, 256)
        self.act1 = nn.Hardswish ()
        self.bn1 = nn.BatchNorm1d(256, momentum=0.9999)

        self.lin2 = nn.Linear(256, 128)
        self.act2 = nn.Hardswish ()
        self.bn2 = nn.BatchNorm1d(128, momentum=0.9999)

        self.lin3 = nn.Linear(128, 64)
        self.act3 = nn.Hardswish()
        self.bn3 = nn.BatchNorm1d(64, momentum=0.9999)

        self.lin4 = nn.Linear(64, 62)

    def forward(self, x):
        x = self.lin1(x)
        x = self.act1(x)
        x = self.bn1(x)
        # x = self.drop1(x)

        x = self.lin2(x)
        x = self.act2(x)
        x = self.bn2(x)
        #x = self.drop2(x)

        x = self.lin3(x)
        x = self.act3(x)
        x = self.bn3(x)
        #x = self.drop3(x)

        x = self.lin4(x)
        return x


In [13]:
class conv(nn.Module):

    def __init__(self):
        super(conv, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 8)
        self.act1 = nn.Hardswish()
        self.pool1 = nn.MaxPool1d(8, 8)
        self.bn1 = nn.BatchNorm1d(16, momentum=0.9999)

        self.conv2 = nn.Conv1d(16, 32, 8)
        self.act2 = nn.Hardswish()
        self.pool2 = nn.MaxPool1d(8, 8)
        self.bn2 = nn.BatchNorm1d(32, momentum=0.9999)


        self.lin1 = nn.Linear(224, 128)
        self.act3 = nn.Hardswish()
        self.bn3 = nn.BatchNorm1d(128, momentum=0.9999)
        
        self.lin2 = nn.Linear(128, 62)



    def forward(self, x):
        x  = self.conv1(x)
        x = self.act1(x)
        x = self.pool1(x)
        x = self.bn1(x)

        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool2(x)
        x = self.bn2(x)

        x = x.view(-1, 224)

        x = self.lin1(x)
        x = self.act3(x)
        x = self.bn3(x)

        x = self.lin2(x)
        return x

In [4]:
model1 = Net()
model1.load_state_dict(torch.load('polar_linear_model.pth', map_location=torch.device('cpu')))
model1.eval()
model1.double()

Net(
  (lin1): Linear(in_features=512, out_features=256, bias=True)
  (act1): Hardswish()
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.9999, affine=True, track_running_stats=True)
  (lin2): Linear(in_features=256, out_features=128, bias=True)
  (act2): Hardswish()
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.9999, affine=True, track_running_stats=True)
  (lin3): Linear(in_features=128, out_features=64, bias=True)
  (act3): Hardswish()
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.9999, affine=True, track_running_stats=True)
  (lin4): Linear(in_features=64, out_features=62, bias=True)
)

In [5]:
model2 = conv()
model2.load_state_dict(torch.load('polar_conv.pth', map_location=torch.device('cpu')))
model2.eval()
model2.double()

conv(
  (conv1): Conv1d(1, 16, kernel_size=(8,), stride=(1,))
  (act1): Hardswish()
  (pool1): MaxPool1d(kernel_size=8, stride=8, padding=0, dilation=1, ceil_mode=False)
  (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.9999, affine=True, track_running_stats=True)
  (conv2): Conv1d(16, 32, kernel_size=(8,), stride=(1,))
  (act2): Hardswish()
  (pool2): MaxPool1d(kernel_size=8, stride=8, padding=0, dilation=1, ceil_mode=False)
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.9999, affine=True, track_running_stats=True)
  (lin1): Linear(in_features=224, out_features=128, bias=True)
  (act3): Hardswish()
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.9999, affine=True, track_running_stats=True)
  (lin2): Linear(in_features=128, out_features=62, bias=True)
)

In [7]:
X = np.zeros(512)

In [8]:
def predict(model, X, conv=False):
    X = X.copy()
    classes = [121, 111, 118, 119, 159, 179, 122, 171, 117, 174, 124, 123, 203,
             158, 173, 216, 206, 181, 185, 188, 225, 182, 211, 232, 205, 286,
             186, 199, 212, 228, 285, 312, 240, 226, 283, 295, 213, 306, 301,
             299, 279, 329, 307, 337, 354, 330, 308, 332, 327, 387, 393, 353,
             338, 394, 390, 302, 389, 344, 340, 396, 416, 448]
    with torch.no_grad():
        X = (X+45.175924783430474)/757.9691984444514
        X = torch.tensor(X).double().unsqueeze(0)
        if conv:
            X = X.unsqueeze(1)
        pred = model(X).numpy()
        pred = list(np.argsort(pred)[0, -20:])[::-1]
        bits = []
    for i in range(len(pred)):
        bits.append(classes[pred[i]])
    return bits

In [9]:
predict(model2, X, True)

[396,
 123,
 344,
 393,
 327,
 394,
 307,
 353,
 188,
 302,
 240,
 213,
 390,
 211,
 206,
 354,
 448,
 389,
 387,
 340]

In [10]:
predict(model1, X)

[387,
 390,
 394,
 389,
 393,
 416,
 448,
 295,
 396,
 330,
 337,
 308,
 353,
 306,
 279,
 329,
 338,
 182,
 283,
 286]

1. BLER по ebN0 для обычного и моделек (по 10k)